In [81]:
import datetime
from flask import Flask, Response
from kafka import KafkaConsumer
from threading import Thread

In [82]:
topic="HoangPhuc-distributed-video1"
kafka_server="kafka:9092"

In [83]:
consumer = KafkaConsumer(
    topic, 
    bootstrap_servers=kafka_server
)

# Set the consumer in a Flask App
app = Flask(__name__)

In [84]:
def get_video_stream():
    """
    Here is where we recieve streamed images from the Kafka Server and convert 
    them to a Flask-readable format.
    """
    for msg in consumer:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpg\r\n\r\n' + msg.value + b'\r\n\r\n')

In [85]:
@app.route('/video', methods=['GET'])
def video():
    """
    This is the heart of our video display. Notice we set the mimetype to 
    multipart/x-mixed-replace. This tells Flask to replace any old images with 
    new values streaming through the pipeline.
    """
    return Response(
        get_video_stream(), 
        mimetype='multipart/x-mixed-replace; boundary=frame')
def run_app():
    app.run(host='0.0.0.0', port=5002, debug=False, use_reloader=False)

# chạy server trong thread riêng
thrd = Thread(target=run_app)
thrd.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://172.19.0.6:5002
Press CTRL+C to quit
